In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15
!pip install transformers

In [ ]:
import tensorflow as tf
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

In [ ]:
import numpy as np
text3sentences = np.load("../input/wikitext4000sentences/text3sentences.npy")

In [ ]:
'''train_data=[]
for i in text.split("."):
    train_data.append(i.strip())'''
print(len(text3sentences))

In [ ]:
#train_data[300:600]
text3sentences

In [ ]:
filer=open("../input/wikitext2tokens/wiki.train.tokens",'r')
print(filer.read())

In [ ]:
sentences=[]
with open("../input/wikitext2tokens/wiki.train.tokens",'r') as f:
    s=""
    temp=f.readline()
    if(temp=="."):
        s+="."
        sentences.append(s)
    else:
        s+=temp
        

In [ ]:
len(sentences)

In [ ]:
#f = open("../input/sentencesfor2fold/10000Sentencesfor2foldAUG.txt",'r')
f = open("../input/gopidataset/printtest.txt",'r')
text = f.read()
lines = text.split("\n")

In [ ]:
len(lines)

In [ ]:
!wget -P /drive/MyDrive/data/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
!pip install gensim
from gensim.models import KeyedVectors
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity 
import tensorflow_hub as hub
import tensorflow as tf
from time import time
import numpy as np
from transformers import pipeline

In [ ]:
!pip install gensim
from gensim.models import KeyedVectors
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity 
import tensorflow_hub as hub
import tensorflow as tf
from time import time
import numpy as np
from transformers import pipeline
#connecting to drive

#Load word2vec (Google)

GOOGLE_NEWS_VECTORS = '/drive/MyDrive/data/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(GOOGLE_NEWS_VECTORS, binary=True)
print("Vectors loaded successfully: "+str(len(word2vec.key_to_index)))

#elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
elmo = hub.Module("/kaggle/input/samsungprismassets/elmo_2", trainable=True)
    
def elmo_vectors(x):
  embeddings=elmo(x, signature="default", as_dict=True)["elmo"]
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

#Load custom elmo embeddings
import pickle
file_to_read1 = open("../input/allcustomelmoembeddings/custom_gpe_final.pickle", "rb")
custom_gpe=pickle.load(file_to_read1)
file_to_read2 = open("../input/allcustomelmoembeddings/custom_org_final.pickle", "rb")
custom_org=pickle.load(file_to_read2)
file_to_read3 = open("../input/allcustomelmoembeddings/custom_per_final.pickle", "rb")
custom_per=pickle.load(file_to_read3)
file_to_read4 = open("../input/allcustomelmoembeddings/customloc.pickle", "rb")
custom_loc=pickle.load(file_to_read4)


In [ ]:
!pip install spacy

In [ ]:
#Dont Run
#import numpy as np
#train_data=np.load("../input/partofsquadforaugumentation/train_data.npy")

In [ ]:
#len(train_data[0:2801])

In [ ]:
import spacy
#sentences=['Who is Sundar Pichai?','Who is the best basketball player in the world?','What day is it today?','My name is Micheal and i live in Chicago','My name is Rahul and i live in Chennai','Why is samsung the best smartphone company?','IIT Bombay is one of Indias top 5 engineering colleges']
nlp= spacy.load('en_core_web_lg')

In [ ]:
def generatedata(entities,org_namedentity,sentence):
    for i in entities:
        sentence2=sentence.replace(org_namedentity,i)
        output=open("Database.txt",'a')
        output.write(sentence2+"\n")
    #print("completed generating data for "+ org_namedentity)

In [ ]:
def customembd(tag):
    if(tag=="NORP"):
        return custom_gpe
    if(tag=="ORG"):
        return custom_org
    if(tag=="PERSON"):
        return custom_per
    if(tag=="LOC"):
        return custom_loc
    else:
        return {}

In [ ]:
#for sentence in train_data[300:600]:
#    print(str(sentence))

In [ ]:
#for ent in nlp(str(train_data[0])).ents:
#    print(str(ent)+" : "+str(ent.label_))
#    try:
#        print(word2vec.most_similar(str(ent),topn=2))
#    except:
#        print(elmo_vectors([str(ent)])[0])
#result=word2vec.most_similar("Beyoncé",topn=2)
#print(result)

In [ ]:
import spacy
sentences=['Who is Sundar Pichai?','Who is the best basketball player in the world?','What day is it today?','My name is Micheal and i live in Chicago','My name is Rahul and i live in Chennai','Why is samsung the best smartphone company?','IIT Bombay is one of Indias top 5 engineering colleges']
nlp= spacy.load('en_core_web_lg')

# NEW Code

In [ ]:
Word2vecSentences=[]
ElmoSentences=[]
word="Newyork"

for sentence in sentences:
    NamedEntities=[]
    for ent in (nlp(str(sentence))).ents:
        NamedEntities.append([sentence[int(ent.start_char):int(ent.end_char)],ent.label_])
    for ent,tag in NamedEntities:
        print(ent)
        Containsinw2v=True
        if ent not in word2vec.key_to_index:
            Containsinw2v=False
            ElmoSentences.append(sentence)
            break
    if(Containsinw2v==True):
        Word2vecSentences.append(sentence)


In [ ]:
words=["Sundar Pichai","today","Micheal","Rahul","samsung","IIT Bombay"]
for word in words:
    if word in word2vec.key_to_index:
        print(word)
    

In [ ]:
sentences[0]

# Working New Code:

In [ ]:
with open("../input/augsents2977-have-to-process/2977SentsforAUGKaggle_Have_to_pass_through_Final_Architechture.txt","r") as f:
    text=f.readlines()
print(len(text))
print(text[0])

In [ ]:
def GenerateData(entities,org_namedentity,sentence):
    for i in entities:
        sentence2=sentence.replace(org_namedentity,i)
        output=open("Database.txt",'a')
        output.write(sentence2+"\n")

In [ ]:
sentences=text

In [ ]:
Allowed_Entities=["NORP","ORG","PERSON","LOC"]
word2vecMappings=[]
elmoMappings=[]
for i in range(0,len(sentences)):
    NamedEntities=[]
    for ent in (nlp(str(sentences[i]))).ents:
        NamedEntities.append([sentences[i][int(ent.start_char):int(ent.end_char)],ent.label_,int(ent.start_char),int(ent.end_char)])
    for j in range(0,len(NamedEntities)):
        #print(NamedEntities[j][1])
        if(NamedEntities[j][1] in Allowed_Entities):
            if NamedEntities[j][0] in word2vec.key_to_index:
                print(NamedEntities[j][0])
                word2vecMappings.append([i,NamedEntities[j][0],NamedEntities[j][2],NamedEntities[j][3],NamedEntities[j][1]])
            else:
                elmoMappings.append([i,NamedEntities[j][0],NamedEntities[j][2],NamedEntities[j][3],NamedEntities[j][1]])
    
    '''
    for j in range(0,len(sentences[i].split(" "))):
        if sentences[i].split(" ")[j] in word2vec.key_to_index:
            print(sentences[i].split(" ")[j])
            word2vecMappings.append([i,j,sentences[i].split(" ")[j]])
        else:
            elmoMappings.append([i,j,sentences[i].split(" ")[j]])
            '''

In [ ]:
Nfold=2

In [ ]:
#word2vec Predictions
for i in range(0,len(word2vecMappings)):
    Preds=word2vec.most_similar(word2vecMappings[i][1],topn=Nfold)
    OnlyWords=[]
    for j in Preds:
        OnlyWords.append(j[0])
    word2vecMappings[i][1]=OnlyWords

In [ ]:
#elmo Predictions
Temp=[]
for i in range(0,len(elmoMappings)):
    Temp.append(elmoMappings[i][1])
start=0
offset=300
while((start+offset)<len(Temp)):
    tempEmbeddings=elmo_vectors(Temp[start:(start+offset)])
    Result=[]
    for Target in range(0,len(tempEmbeddings)):
        temp=[]
        temp2=[]
        temp3=customembd(elmoMappings[start+Target][4])
        for i,j in temp3.items():
            temp.append(i)
            temp2.append(j)
        print(elmoMappings[start+Target][4])
        Predictions=[]
        for i in range(Nfold):
            vectors=np.array(temp2)
            target=tempEmbeddings[Target]
            start_time = time()
            myvals = np.dot(vectors, target)
            max_index = np.argmax(myvals)
            max_similarity = myvals[max_index]
            #print("done with max dot in %s seconds" % (time() - start_time))
            #print("Most similar vector to target is index %s with %s" % (max_index, max_similarity))
            count=0 
            targetcount=max_index
            for p in temp:
                if(count==targetcount):
                    Predictions.append(p)
                    temp.pop(targetcount)
                    temp2.pop(targetcount)
                count=count+1
        #elmoMappings[Target][1]=Predictions
    #print(elmoMappings)
    #Generating Elmo generated sentences
        sentNum=elmoMappings[start+Target][0]
        Predicted=Predictions
        OrgWord=sentences[sentNum][elmoMappings[start+Target][2]:elmoMappings[start+Target][3]]
        print(OrgWord+" : ")
        print(Predicted)
        GenerateData(Predicted,OrgWord,sentences[sentNum])
    start+=offset

In [ ]:
#Generating Word2Vec generated sentences
for i in range(0,len(word2vecMappings)):
    sentNum=word2vecMappings[i][0]
    Predicted=word2vecMappings[i][1]
    OrgWord=sentences[sentNum][word2vecMappings[i][2]:word2vecMappings[i][3]]
    GenerateData(Predicted,OrgWord,sentences[sentNum])

# OLD Code

In [ ]:
DebugCount=1
nfold = 3
for sentence in ["Riela is walking"]:
    Counter=0
    NamedEntities=[]
    for ent in (nlp(str(sentence))).ents:
        NamedEntities.append([sentence[int(ent.start_char):int(ent.end_char)],ent.label_])
    #Code for DataAugumentation:
    #print("sentence: "+(str)(len(NamedEntities)))
    
    for i in range(0,len(NamedEntities)):
        if(Counter<nfold):
            Counter+=1
            NamedEntity=NamedEntities[i]
            try:
                if((NamedEntity[1]!="NORP")and(NamedEntity[1]!="ORG")):
                    result=word2vec.most_similar(NamedEntity[0],topn=2)
                    Result=[]
                    for i in result:
                        Result.append(i[0])
                        print(Result)
                    generatedata(Result,NamedEntity[0],sentence)
                    print(Result)
                else:
                    if((NamedEntity[1]=="NORP")or(NamedEntity[1]=="ORG")or(NamedEntity[1]=="PERSON")or(NamedEntity[1]=="LOC")):
                        Result=[]
                        targetarray=elmo_vectors([NamedEntity[0]])[0]
                        temp=[]
                        temp2=[]
                        temp3=customembd(NamedEntity[1])
                        for i,j in temp3.items():
                            temp.append(i)
                            temp2.append(j)
                        for i in range(2):
                            vectors=np.array(temp2)
                            target=targetarray
                            start_time = time()
                            myvals = np.dot(vectors, target)
                            max_index = np.argmax(myvals)
                            max_similarity = myvals[max_index]
                            #print("done with max dot in %s seconds" % (time() - start_time))
                            #print("Most similar vector to target is index %s with %s" % (max_index, max_similarity))
                            count=0 
                            targetcount=max_index
                        for i,j in temp3.items():
                            if(count==targetcount):
                                Result.append(i)
                            count=count+1
                            temp.pop(targetcount)
                            temp2.pop(targetcount)
                        generatedata(Result,NamedEntity[0],sentence)
                        print(Result)
            except:
                if((NamedEntity[1]=="NORP")or(NamedEntity[1]=="ORG")or(NamedEntity[1]=="PERSON")or(NamedEntity[1]=="LOC")):
                    Result=[]
                    targetarray=elmo_vectors([NamedEntity[0]])[0]
                    temp=[]
                    temp2=[]
                    temp3=customembd(NamedEntity[1])
                    for i,j in temp3.items():
                        temp.append(i)
                        temp2.append(j)
                    for i in range(2):
                        vectors=np.array(temp2)
                        target=targetarray
                        start_time = time()
                        myvals = np.dot(vectors, target)
                        max_index = np.argmax(myvals)
                        max_similarity = myvals[max_index]
                        #print("done with max dot in %s seconds" % (time() - start_time))
                        #print("Most similar vector to target is index %s with %s" % (max_index, max_similarity))
                        count=0 
                        targetcount=max_index
                        for p,j in temp3.items():
                            if(count==targetcount):
                                Result.append(p)
                            count=count+1
                        temp.pop(targetcount)
                        temp2.pop(targetcount)
                    generatedata(Result,NamedEntity[0],sentence)
                    print(Result)
    print(DebugCount)
    print(Result)
    DebugCount+=1

In [ ]:
while(True):
    pass

In [ ]:
output=open("./Database.txt",'r')
output2=output.read()
sentencesinDB=output2.split("\n")

In [ ]:
len(sentencesinDB)

In [ ]:
for i in nlp(sentencesinDB[0]).ents:
    print(i)

In [ ]:
file=open("FinalDatabase_Gopi_300_3.txt",'w')
def create_pipeline(model_name):
    return pipeline('fill-mask', model=model_name)
predict_mask = create_pipeline("bert-large-cased")
for i in sentencesinDB:
    changed=False
    n=0
    doc=nlp(i)
    for token in doc:
        if(token.pos_=="VERB" or token.pos_=="ADJ"):
            changed=True
            tokstr=str(token)
            berttext=i.replace(tokstr,"[MASK]",1)
            ele = predict_mask(berttext)[0]
            if(ele["token_str"]!=tokstr):
                print(ele["sequence"])
                file.write(ele["sequence"])
                file.write("\n")
                n+=1
    if(changed==False):
        file.write(i+"\n")

**End of the Code**

In [ ]:
while(True):
    pass

In [ ]:
!pip install language_tool_python

In [ ]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')
# get the matches


In [ ]:
count=1
file2=open("FinalDatabase.txt",'r')
content=file2.read()
for i in content.split("\n"):
    matches = tool.check(i)
    #print(str(count)+": ")
    #print(len(matches))
    #print(matches)
    for j in matches:
        count2=1
        for k in j:
            if(count2!=1):
                break
            else:
                if(k=="ENGLISH_WORD_REPEAT_RULE"):
                    print(i)
            count2=count2-1
    count=count+1


In [ ]:
!python -m pip install ../input/languagecheckermodule/language-check-1.1

In [ ]:
!rm Database.txt

# **Training the transformer based Language Model**

In [ ]:
!git clone https://github.com/pytorch/fairseq
!cd fairseq
!pip install --editable /kaggle/working/

In [ ]:
!git clone https://github.com/NVIDIA/apex
!cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" \
  --global-option="--deprecated_fused_adam" --global-option="--xentropy" \
  --global-option="--fast_multihead_attn" ./

In [ ]:
!pip install fairseq

In [ ]:
cd fairseq/

In [ ]:
!fairseq-preprocess \
--only-source \
--trainpref /kaggle/input/wikitext2tokens/wiki.train.tokens \
--validpref /kaggle/input/wikitext2tokens/wiki.valid.tokens \
--testpref /kaggle/input/wikitext2tokens/wiki.test.tokens \
--destdir data-bin/summary \
--workers 20

In [ ]:
CUDA_VISIBLE_DEVICES=0 
!fairseq-train --task language_modeling \
data-bin/summary \
--save-dir checkpoints/transformer_summary \
--arch transformer_lm --share-decoder-input-output-embed \
--dropout 0.1 \
--optimizer adam --adam-betas '(0.9, 0.98)' --weight-decay 0.01 --clip-norm 0.0 \
--lr 0.0005 --lr-scheduler inverse_sqrt --warmup-updates 4000 --warmup-init-lr 1e-07 \
--tokens-per-sample 512 --sample-break-mode none \
--max-tokens 2048 --update-freq 16 \
--fp16 \
--max-update 50000 \
--max-epoch 12

In [ ]:
!fairseq-eval-lm data-bin/summary \
--path /kaggle/input/checkpointsfortransformerlanguagemodel/checkpoint_best.pt \
--tokens-per-sample 50 \
--context-window 100

In [ ]:
while True:
    pass

# Preprocessing the FinalDatabase.txt file


In [ ]:
output=open("./FinalDatabase_Gopi_300_3.txt",'r')
output2=output.read()
sentencesinDB=output2.split("\n")

In [ ]:
len(sentencesinDB)

In [ ]:
import random
random.shuffle(sentencesinDB)

In [ ]:
outputFinal = open("./FinalDatabase5000_2.txt",'w')
outputFinal.writelines(sentencesinDB[0:5000])
outputFinal.close()

In [ ]:
filer = open("../input/wikitext2tokens/wiki.train.tokens",'r')
texter = filer.read()

# Using ASR for Evaluating LM

In [ ]:
!pip install pandas
!pip install tqdm
!pip install simpletransformers
!pip install lm-scorer
!pip install nemo_toolkit[all]==0.11.0
!pip install torch==1.5.1
!pip install torchaudio==0.5.1
!pip install torchvision==0.6.0
!pip install transformers==2.11.0
!pip install tokenizers==0.7.0

In [ ]:
!pip install nemo==1.3.0

In [ ]:
import argparse
import json
import os

import nemo
import nemo.collections.asr as nemo_asr
import torch
from nemo.collections.asr.models import ASRConvCTCModel

parser = argparse.ArgumentParser(description=__doc__)

parser.add_argument("wav_path", type=str, default="./Welcome.wav",
                    help="Path to wav file with the sound for transcription")

parser.add_argument("am_path", type=str, default="JasperNet10x5-En-Base.nemo",
                    help="Path to NeMo acoustic model")

parser.add_argument("lm_path", type=str, default="common_crawl_00.prune01111.trie.klm",
                    help="Path to KenLM N-Gram language model")

parser.add_argument("--batch_size", type=int, default=32,
                    help="Batch size for acoustic model")

parser.add_argument("--beam_size", type=int, default=2048,
                    help="Beam size for beam search with language model")

parser.add_argument("--alpha", type=float, default=2.5,
                    help="Alpha parameter for beam search")

parser.add_argument("--beta", type=float, default=0.3,
                    help="Beta parameter for beam search")

parser.add_argument("--output_path", type=str, default="candidates.txt",
                    help="Path to KenLM N-Gram language model")

args = parser.parse_args()


def recognize_speech():
    if torch.cuda.is_available():
        neural_factory = nemo.core.NeuralModuleFactory(
            placement=nemo.core.DeviceType.GPU
            , optimization_level=nemo.core.Optimization.mxprO1
        )
    else:
        neural_factory = nemo.core.NeuralModuleFactory(
            placement=nemo.core.DeviceType.CPU)

    # noinspection PyTypeChecker
    asr_model: ASRConvCTCModel = nemo_asr.models.ASRConvCTCModel.from_pretrained(model_info=args.am_path)
    asr_model.eval()

    beam_search_with_lm = nemo_asr.BeamSearchDecoderWithLM(
        vocab=asr_model.vocabulary,
        beam_width=args.beam_size,
        alpha=args.alpha,
        beta=args.beta,
        lm_path=args.lm_path,
        num_cpus=max(os.cpu_count(), 1))

    # Create dummy manifest with single file
    manifest_path = "manifest.transcription"
    with open(manifest_path, 'w') as f:
        f.write(json.dumps({"audio_filepath": args.wav_path, "duration": 18000, "text": "todo"}))

    data_layer = nemo_asr.AudioToTextDataLayer(
        shuffle=False,
        manifest_filepath=manifest_path,
        labels=asr_model.vocabulary, batch_size=args.batch_size)

    audio_signal, audio_signal_len, _, _ = data_layer()

    log_probs, encoded_len = asr_model(input_signal=audio_signal, length=audio_signal_len)
    beam_predictions = beam_search_with_lm(
        log_probs=log_probs, log_probs_length=encoded_len)
    eval_tensors = [beam_predictions]

    tensors = neural_factory.infer(tensors=eval_tensors, use_cache=False, cache=False, offload_to_cpu=True)

    batch = tensors[-1][0]
    prediction = batch[0]
    candidates = [candidate[1] for candidate in prediction]

    with open(args.output_path, 'w') as f:
        for candidate in candidates:
            f.write(candidate + "\n")